## Coronavirus disease 19
Coronavirus disease 19(COVID-19) is very contagious, pathogenic viral infection. Below in figure 1,2,3,4 are data about the current circumstances of COVID-19 in the UK according to Public Health England.  In figure 1, there is a graph representing the current cases, hospital admissions and deaths related to COVID-19. In figure 2 is a breakdown of cumulative infection totals between the two genders and the total amount. Figure 3 indicates the cumulative conducted COVID-19 tests in relation to the cumulative capacity of test analysis that can occur in the UK that day. Figure 4 represents the amount of all new cases against the cumulative test conducted that day.

Created by Rohimur Rohman and all data based on UK Government data published by Public Health England.

In [ ]:
#print('Coronavirus disease 19(COVID-19) is very contagious, pathogenic viral infection. Below in figure 1,2,3,4 are data about the current circumstances of COVID-19 in the UK according to NHS data.  In figure 1, there is a graph representing the current cases, hospital admission and deaths related to COVID-19. In figure 2 is a breakdown of cumulative infection total between the two genders and the total amount. Figure 3 indicates the cumulative conducted COVID-19 tests in relation to the cumulative capacity of test analysis that can occur in the UK that day. Figure 4 represents the amount of all new cases against the cumulative test conduct that day.')
#please use above print if everything needs to be on one cell
from uk_covid19 import Cov19API
import json
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API
jsondata={}

def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    apidata=access_api()
    global df
    df=wrangle_data(apidata) # wrangle the data and overwrite the dataframe for plotting
    refresh_graph()
    apibutton.icon="check"
    
def access_api():
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
    return {} # return data read from the API

def wrangle_data(rawdata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    df=pd.DataFrame(index=range(0,100), columns=['One', 'Two'])
    one=two=0.0
    for i in range(0,100):
        df.loc[i,'One']=one
        df.loc[i,'Two']=two
        one+=np.random.randn()
        two+=2*np.random.randn()
    return df

df=wrangle_data(jsondata) # df is the dataframe for plotting

def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

def min_age(agerange):
    agerange=agerange.replace('+','') # remove the + from 90+
    start=agerange.split('_')[0]
    return int(start)

def access_api(button):
    apibutton.icon="check"
    apibutton.disabled=True

def age_graph(graphcolumns):# our callback function.
    ncols=len(graphcolumns)
    if ncols>0:
        age_df.plot(kind='bar', y=list(graphcolumns)) # forms a list
        plt.show() # graphs won't update properly if this is missing
    else:
        # if the user has not selected any column, print a message instead
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

def timeseries_graph(gcols,gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        timeseriesdf[list(gcols)].plot(logy=logscale)
        plt.show() # graphs won't update if this is missing 
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

        
#------------------------------------------Filters----------------------------------------------

#filter for case/hospital/deaths #GM
filters=['areaType=overview'] # area filters
structure={"date": "date","cases": "newCasesByPublishDate","hospital": "newAdmissions","deaths": "cumDeaths28DaysByDeathDateRate"}#gains data from NHS 
api=Cov19API(filters=filters, structure=structure)
timeseries=api.get_json()
#print(timeseries)

#filter gender/age in england #GM
filters=['areaType=nation','areaName=England']
structure={"males": "maleCases","females": "femaleCases"} 
api=Cov19API(filters=filters, structure=structure)
agedistribution=api.get_json()
#print(agedistribution)

# filter testtaken/testing cap p#GM
filters=['areaType=overview']
structure={'date':'date','TestThatDay':'newTestsByPublishDate','TestCapDay':'plannedPCRCapacityByPublishDate'}
api=Cov19API(filters=filters, structure=structure)
testcap=api.get_json()
#print(testcap)

#newcases and all c-19 tests #GM
filters=['areaType=overview']
structure={'date':'date','NewCases':'newCasesByPublishDate','TestThatDay':'newTestsByPublishDate'}
api=Cov19API(filters=filters, structure=structure)
NewctoTestc=api.get_json()
#print(NewctoTestc)


############------------------------------Graphs------------------------------------------------------------
########-----case/hospital/deaths
with open("timeseries.json", "wt") as OUTF:
    json.dump(timeseries, OUTF)
    
with open("timeseries.json", "rt") as INFILE:
    data=json.load(INFILE)
    
datalist=data['data']# NHS data in json format/list of dic's


dates=[dictionary['date'] for dictionary in datalist ]
dates.sort() # sort dates out, just incase they are not sorted

startdate=parse_date(dates[0])
enddate=parse_date(dates[-1]) # dates sorted

index=pd.date_range(startdate, enddate, freq='D')#makes dates into x axis
timeseriesdf=pd.DataFrame(index=index, columns=['cases', 'hospital', 'deaths']) # put data from NHS into dataframe


for entry in datalist: # each entry is a dictionary with date, cases, hospital and deaths
    date=parse_date(entry['date'])
    for column in ['cases', 'hospital', 'deaths']:
        if pd.isna(timeseriesdf.loc[date, column]): 
            value= float(entry[column]) if entry[column]!=None else 0.0 # replace None with 0 in our data 
            timeseriesdf.loc[date, column]=value
            
timeseriesdf.fillna(0.0, inplace=True)# fill nulls with 0.0
            

timeseriesdf.to_pickle("timeseriesdf.pkl")#saves to allow manipulation in a dashboard

apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='',
    tooltip='Click to download current Public Health England data',
    icon='download')

# register the callback function with the button
apibutton.on_click(access_api)

# display the widgets
display(apibutton)

timeseriesdf=pd.read_pickle("timeseriesdf.pkl")

series=wdg.SelectMultiple(
    options=['cases', 'hospital', 'deaths'],
    value=['cases', 'hospital', 'deaths'],
    rows=3,
    description='Stats:',
    disabled=False)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False)

# keep calling timeseries_graph(gcols=value_of_series, gscale=value_of_scale); 
# capture output in widget graph   
graph=wdg.interactive_output(timeseries_graph, {'gcols': series, 'gscale': scale})

# stack series and scale on top of each other
ctrls=wdg.VBox([series, scale])
# put the graph and the controls side by side
form=wdg.HBox([graph, ctrls])
display(form)
print('Figure 1- cases = the number of new cases that day, hospital = the number of new hospital admissions that day, deaths = the cumulative death rate in the past 28 days. Also included is a log scaler and the ability to view a single variable or multiple for comparison.')
######################-------------------age-gender---------------------------------------------------
with open("agedistribution.json", "wt") as OUTF:
    json.dump(agedistribution, OUTF)

with open("agedistribution.json", "rt") as INFILE:
    data=json.load(INFILE)
    
datadic=data['data'][0] # data['data'] is a list

males=datadic['males']
females=datadic['females']

ageranges=[x['age'] for x in males] # each entry of males is a dictionary

ageranges.sort(key=min_age)
age_df=pd.DataFrame(index=ageranges, columns=['males','females', 'total'])

for entry in males: # each entry is a dictionary
    ageband=entry['age'] # our index position
    age_df.loc[ageband, 'males']=entry['value']
    
for entry in females:
    ageband=entry['age']
    age_df.loc[ageband, 'females']=entry['value']
    
age_df['total']=age_df['males']+age_df['females']

age_df.to_pickle("agedf.pkl")

age_df=pd.read_pickle("agedf.pkl")

agecols=wdg.SelectMultiple(
    options=['males', 'females', 'total'], # options available
    value=['males', 'females'], # initial value
    rows=3, # rows of the selection box
    description='Sex',
    disabled=False)

    
# keep calling age_graph(graphcolumns=value_of_agecols); capture output in widget output    
output=wdg.interactive_output(age_graph, {'graphcolumns': agecols})

display(agecols, output)
print('Figure 2- Is a bar chart, which shows the number of males and females in different age groups, that have contracted covid-19 cumulatively according to NHS records. Also included is the ability to see the total amount of cases and ability to compare a single gender to the total.')
###########----- testthatay/testcap

with open("testcap.json", "wt") as OUTF:
    json.dump(testcap, OUTF)
with open("testcap.json", "rt") as INFILE:
    data=json.load(INFILE)
    

datalist2=data['data']
#print(datalist2)
dates2=[dictionary['date'] for dictionary in datalist2 ]
dates2.sort()

startdate=parse_date(dates[0])
enddate=parse_date(dates[-1])

index=pd.date_range(startdate, enddate, freq='D')
testcap=pd.DataFrame(index=index, columns=['TestThatDay', 'TestCapDay'])

for entry in datalist2: #put data into dataframe
    date=parse_date(entry['date'])
    for column in ['TestThatDay', 'TestCapDay']:
        if pd.isna(testcap.loc[date, column]): 
            value= float(entry[column]) if entry[column]!=None else 0.0
            testcap.loc[date, column]=value
            
testcap.fillna(0.0, inplace=True)
#print(testcap)

testcap.to_pickle("testcap.pkl")

#testcap.plot() 
#testcap.plot(logy=True) 
series2=wdg.SelectMultiple(
    options=['TestThatDay', 'TestCapDay'],
    value=['TestThatDay', 'TestCapDay'],
    rows=2,
    description='Stats:',
    disabled=False)

scale2=wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False)

def testcap_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        testcap[list(gcols)].plot(logy=logscale)
        plt.show()# graphs won't update if this is missing 
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
        
graph2=wdg.interactive_output(testcap_graph, {'gcols': series2, 'gscale': scale2})
ctrls2=wdg.VBox([series2, scale2])
# put the graph and the controls side by side
form2=wdg.HBox([graph2, ctrls2])
display(form2)
print('Figure 3- Is an interactive line graph showing the total number of covid-19 tests conducted that day and test analysis capacity that day. TestThatDay = total number of covid-19 tests conducted that day.  TestCapDay = test analysis capacity that day. Also included is a log scaler and the ability to view each variable on its own.')
###########--------newcase/testthatday

with open("NewctoTestc.json", "wt") as INFILE:
    json.dump(NewctoTestc, INFILE)
with open("NewctoTestc.json", "rt") as INFILE:
    data=json.load(INFILE)

datalist3=data['data']
#print(datalist3)
dates3=[dictionary['date'] for dictionary in datalist3 ]
dates3.sort()

startdate=parse_date(dates[0])
enddate=parse_date(dates[-1])

index=pd.date_range(startdate, enddate, freq='D')
NewctoTestc=pd.DataFrame(index=index, columns=['NewCases', 'TestThatDay'])

for entry in datalist3: 
    date=parse_date(entry['date'])
    for column in ['NewCases', 'TestThatDay']:
        if pd.isna(NewctoTestc.loc[date, column]): 
            value= float(entry[column]) if entry[column]!=None else 0.0
            NewctoTestc.loc[date, column]=value
            
NewctoTestc.fillna(0.0, inplace=True)

#NewctoTestc.plot() 
#NewctoTestc.plot(logy=True) 

NewctoTestc.to_pickle("NewctoTestc.pkl")

series3=wdg.SelectMultiple(
    options=['NewCases', 'TestThatDay'],
    value=['NewCases', 'TestThatDay'],
    rows=2,
    description='Stats:',
    disabled=False)

scale3=wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False)

def NewctoTestc_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        NewctoTestc[list(gcols)].plot(logy=logscale)
        plt.show() # graphs won't update if this is missing 
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
        
graph3=wdg.interactive_output(NewctoTestc_graph, {'gcols': series3, 'gscale': scale3})
# stack series and scale on top of each other
ctrls3=wdg.VBox([series3, scale3])
# put the graph and the controls side by side
form3=wdg.HBox([graph3, ctrls3])
display(form3)
print('Figure 4- Is an interactive line graph that shows the total number of new cases that day and test taken that day. NewCases = number of new cases that day. TestThatDay = number of covid tests taken that day. Also included is a log scaler and the ability to see view variable on its own.')